# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 9:21AM,248021,10,0086057490,ISDIN Corporation,Released
1,Oct 6 2022 9:21AM,248021,10,0086057505,ISDIN Corporation,Released
2,Oct 6 2022 9:21AM,248021,10,0086057511,ISDIN Corporation,Released
3,Oct 6 2022 9:21AM,248021,10,0086057521,ISDIN Corporation,Released
4,Oct 6 2022 9:21AM,248021,10,0086057519,ISDIN Corporation,Released
5,Oct 6 2022 9:21AM,248021,10,0086057525,ISDIN Corporation,Released
6,Oct 6 2022 9:21AM,248021,10,0086057536,ISDIN Corporation,Released
7,Oct 6 2022 9:21AM,248021,10,0086057540,ISDIN Corporation,Released
8,Oct 6 2022 9:21AM,248021,10,0086057539,ISDIN Corporation,Released
9,Oct 6 2022 9:21AM,248021,10,0086057538,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,248005,Released,1
14,248014,Released,6
15,248015,Released,14
16,248018,Released,28
17,248021,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248005,NaN,NaN,1.0
248014,NaN,NaN,6.0
248015,NaN,NaN,14.0
248018,NaN,NaN,28.0
248021,NaN,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247926,13.0,0.0,2.0
247964,23.0,11.0,1.0
247972,4.0,1.0,9.0
247983,0.0,8.0,6.0
247985,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247926,13,0,2
247964,23,11,1
247972,4,1,9
247983,0,8,6
247985,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247926,13,0,2
1,247964,23,11,1
2,247972,4,1,9
3,247983,0,8,6
4,247985,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247926,13,,2
1,247964,23,11,1
2,247972,4,1,9
3,247983,,8,6
4,247985,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation
23,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation
51,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.
52,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC"
66,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd"
72,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation
90,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC"
91,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC"
94,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd"
108,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,,,23
1,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,,,28
2,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,,,1
3,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC",,,14
4,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",,,6
5,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation,,,18
6,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC",,,1
7,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC",,,3
8,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",,8,6
9,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation,4,1,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,23,,
1,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,28,,
2,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,,
3,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC",14,,
4,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,,
5,Oct 5 2022 3:19PM,247993,10,ISDIN Corporation,18,,
6,Oct 5 2022 3:16PM,247991,10,"Snap Medical Industries, LLC",1,,
7,Oct 5 2022 2:31PM,247985,10,"Virtue Rx, LLC",3,,
8,Oct 5 2022 2:19PM,247983,19,"AdvaGen Pharma, Ltd",6,8,
9,Oct 5 2022 1:45PM,247972,10,ISDIN Corporation,9,1,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,23,,
1,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,28,,
2,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,,
3,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC",14,,
4,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,23,NaN,NaN
1,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,28,NaN,NaN
2,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,NaN,NaN
3,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC",14,NaN,NaN
4,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 9:21AM,248021,10,ISDIN Corporation,23,0.0,0.0
1,Oct 6 2022 9:01AM,248018,10,ISDIN Corporation,28,0.0,0.0
2,Oct 6 2022 8:30AM,248005,10,Eywa Pharma Inc.,1,0.0,0.0
3,Oct 6 2022 8:08AM,248015,10,"Snap Medical Industries, LLC",14,0.0,0.0
4,Oct 6 2022 8:06AM,248014,19,"AdvaGen Pharma, Ltd",6,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1984000,97,1.0,4.0
15,247926,2,0.0,13.0
19,743961,13,19.0,23.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1984000,97,1.0,4.0
1,15,247926,2,0.0,13.0
2,19,743961,13,19.0,23.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,97,1.0,4.0
1,15,2,0.0,13.0
2,19,13,19.0,23.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,15,Released,2.0
2,19,Released,13.0
3,10,Executing,1.0
4,15,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19
Status,,,
Completed,4.0,13.0,23.0
Executing,1.0,0.0,19.0
Released,97.0,2.0,13.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19
0,Completed,4.0,13.0,23.0
1,Executing,1.0,0.0,19.0
2,Released,97.0,2.0,13.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19
0,Completed,4.0,13.0,23.0
1,Executing,1.0,0.0,19.0
2,Released,97.0,2.0,13.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()